# Sentiment analysis

In [ ]:
import torch
from torch.optim import AdamW
from transformers import BertTokenizer, BertForSequenceClassification, get_linear_schedule_with_warmup, BertConfig

from torch.utils.data import TensorDataset, DataLoader

import json

# Load the data from JSON file
with open("sentiment_analysis.json", "r") as f:
    data = json.load(f)

# Define the training data
train_data = []
# sub_labels = {}
# for top_level_label, sub_level_labels in data.items():
#     for sub_level_label_data in sub_level_labels:
#         text = sub_level_label_data["text"]
#         sub_level_label = sub_level_label_data["sub_level_label"]
#         train_data.append((text, top_level_label, sub_level_label))
#         if top_level_label not in sub_labels:
#             sub_labels[top_level_label] = []
#         sub_labels[top_level_label].append(sub_level_label)

intent_subintent_map = {}
subintent_labels = set()
for intent, examples in data.items():
    for example in examples:
        subintent_label = example['sub_level_label']
        subintent_labels.add(subintent_label)
        intent_subintent_map[(intent, subintent_label)] = len(subintent_labels) - 1

print(intent_subintent_map)

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# convert the input data to suitable input format for the BERT model
inputs = []
labels = []
for intent, examples in data.items():
    for example in examples:
        text = example['text']
        subintent_label = example['sub_level_label']
        input_ids = tokenizer.encode(text, add_special_tokens=True)
        inputs.append(input_ids)
        labels.append(intent_subintent_map[(intent, subintent_label)])


num_epochs = 10

# load the BERT model configuration and tokenizer
model_config = BertConfig.from_pretrained('bert-base-uncased', num_labels=len(subintent_labels))
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# create the BERT model
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', config=model_config)

# set up the optimizer and learning rate scheduler
optimizer = AdamW(model.parameters(), lr=5e-5, eps=1e-8)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=len(inputs) * num_epochs)

# convert the input data to PyTorch tensors
inputs = torch.tensor(inputs)
labels = torch.tensor(labels)

# create a PyTorch dataset and data loader
dataset = TensorDataset(inputs, labels)
dataloader = DataLoader(dataset, batch_size=24)

# train the BERT model
model.train()
for epoch in range(10):
    for batch in dataloader:
        # get the input and label data for this batch
        batch_inputs, batch_labels = batch

        # zero out the gradients from the previous batch
        optimizer.zero_grad()

        # forward pass through the BERT model and compute the loss
        outputs = model(batch_inputs, labels=batch_labels)
        loss = outputs.loss

        # perform backward pass and update the parameters
        loss.backward()
        optimizer.step()
        scheduler.step()


# Define the mapping between top-level labels and integers
# top_level_label_map = {label: i for i, label in enumerate(set([data[1] for data in train_data]))}
# print(top_level_label_map)

# Define the mapping between sub-level labels and integers
# sub_level_label_map = {sub_label: i for i, sub_label in enumerate(set([sub_label for sub_labels_list in sub_labels.values() for sub_label in sub_labels_list]))}
# print(sub_level_label_map)

# Convert the training data labels to integers using the label_map and sub_label_map
# A tensor is a multi-dimensional array that looks like a numpy array, it's used for neural networks
# top_level_labels = torch.tensor([top_level_label_map[data[1]] for data in train_data])
# sub_level_labels = torch.tensor([sub_level_label_map[sub_label] for data in train_data for sub_label in sub_labels[data[1]]])

# # Load the pre-trained BERT model and tokenizer
# model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=len(top_level_label_map))
# tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# # Tokenize the training data and convert to tensors
# inputs = tokenizer.batch_encode_plus([data[0] for data in train_data], padding=True, truncation=True, return_tensors="pt")

# # Fine-tune the model on the training data
# optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)
# loss_fn = torch.nn.CrossEntropyLoss()
# for epoch in range(10):
#     optimizer.zero_grad()
#     outputs = model(inputs["input_ids"], attention_mask=inputs["attention_mask"], labels=top_level_labels)
#     loss = outputs.loss
#     loss.backward()
#     optimizer.step()
    

#     print(f"Epoch {epoch+1}, Loss: {loss.item()}")

#     # Evaluate the model on the training data
#     predictions = outputs.logits.argmax(axis=1)
#     accuracy = (predictions == top_level_labels).sum()


In [ ]:
# Test the model

import torch

def predict_intent_subintent(input_text):
    # Preprocess the input text
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    input_tokens = tokenizer(input_text, padding=True, truncation=True, return_tensors='pt')
    input_ids = input_tokens['input_ids'].to(device)
    attention_mask = input_tokens['attention_mask'].to(device)

    # Wrap the input in a dictionary
    inputs = {'input_ids': input_ids, 'attention_mask': attention_mask}

    # Set the model to evaluation mode
    model.eval()

    # Pass the input to the model and get the output logits
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits

    # Get the index of the highest scoring logits as the predicted label
    predicted_label_index = torch.argmax(logits, dim=1).item()

    # Map the predicted label to the corresponding intent and sub-intent using the intent_subintent_map
    predicted_intent, predicted_subintent = intent_subintent_map[predicted_label_index]

    # Return the predicted intent and sub-intent
    return predicted_intent, predicted_subintent


# def predict_intent(text):

#     top_level_inputs = tokenizer.encode_plus(text, padding=True, truncation=True, return_tensors="pt")
#     top_level_outputs = model(top_level_inputs["input_ids"], attention_mask=top_level_inputs["attention_mask"])
#     # top_level_predicted_labels = torch.argsort(top_level_outputs.logits, descending=True).tolist()[0]
#     top_level_predicted_label = torch.argmax(top_level_outputs.logits).item()
#     top_level_predicted_intent = [k for k, v in top_level_label_map.items() if v == top_level_predicted_label]
    
#     sub_level_inputs = tokenizer.encode_plus(text, padding=True, truncation=True, return_tensors="pt")
#     sub_level_outputs = model(sub_level_inputs["input_ids"], attention_mask=sub_level_inputs["attention_mask"])
#     sub_level_predicted_label = torch.argmax(sub_level_outputs.logits).item()
#     # sub_level_predicted_intents = [k for k, v in sub_level_label_map.items() if v in sub_level_predicted_labels]

#     print(sub_level_label_map)

#     sub_level_predicted_intent = [k for k, v in sub_level_label_map.items() if v == sub_level_predicted_label]
    
    
#     print("predicted sublevel intents", sub_level_predicted_intent)

#     return top_level_predicted_intent[0] if top_level_predicted_intent else None, sub_level_predicted_intent[0] if sub_level_predicted_intent else None
    

top_level_intents, sub_level_intents = predict_intent("What information does the privacy policy collect?")
print(top_level_intents, sub_level_intents)


In [ ]:
import gradio as gr
import time
import random

# load the pre-trained intent analysis model
# nlp = spacy.load("en_trf_bertbaseuncased_lg")

response_map = {
    ("security", "security_relating_to"): ["Our security measures include...", "We take security very seriously and have implemented..."],
    ("security", "security_concerns"): ["We understand your security concerns and have taken steps to address them.", "You can trust that your information is safe with us."],
    ("information", "information_about"): ["Our store offers a variety of products, including...", "We also have a rewards program that allows you to earn points on your purchases."],
    ("information", "information_schedule"): ["We are open from 9am to 10pm, 7 days a week.", "Our business hours are 9am to 5pm, Monday to Friday."],
    ("help", "help_with_finding"): ["Here are some hotels near the airport:...", "I can help you find a hotel that meets your needs."],
    ("help", "help_with_booking"): ["You can book a room on our website or by calling our reservation hotline.", "We also offer a loyalty program that gives you discounts on future bookings."],
    ("information", "ordering"): ["You can place an order on our website or by calling our order hotline.", "We also offer a loyalty program that gives you discounts on future orders."]
}

with gr.Blocks() as demo:
    chatbot = gr.Chatbot()
    msg = gr.Textbox()
    clear = gr.Button("Clear")

    def user(user_message, history):
        return "", history + [[user_message, None]]

    def bot(history):
        user_message = history[-1][0]

        intent = predict_intent(text=user_message)
        print('intent:', intent)
        # Random choice randomly chooses one of the options that matches the intent
     
        # generating a response with GPT if the main intent was 'privacy_policy' or 'legal_statement'
        use_gpt = intent[0] == 'privacy_policy' or intent[0] == 'legal_statement'

        response = gpt_model.answer_question(question=user_message) if use_gpt else 'No idea, bitch'

        # response = random.choice(response_map[intent])
        history[-1][1] = response
        # The sleep is to simulate a more natural conversation
        time.sleep(1)
        return history

    msg.submit(user, [msg, chatbot], [msg, chatbot], queue=False).then(
        bot, chatbot, chatbot
    )
    clear.click(lambda: None, None, chatbot, queue=False)

demo.launch()


## GPT

In [ ]:
# GPT model herel 
%load_ext autoreload
%autoreload 2

from gpt import GPT

gpt_model = GPT()

#gpt_model.answer_question(question='What is the most important thing I need to know about your privacy statement?')


In [ ]:
# GPT query function here


In [ ]:
import gradio as gr
import time
import random

# load the pre-trained intent analysis model
# nlp = spacy.load("en_trf_bertbaseuncased_lg")

response_map = {
    ("security", "security_relating_to"): ["Our security measures include...", "We take security very seriously and have implemented..."],
    ("security", "security_concerns"): ["We understand your security concerns and have taken steps to address them.", "You can trust that your information is safe with us."],
    ("information", "information_about"): ["Our store offers a variety of products, including...", "We also have a rewards program that allows you to earn points on your purchases."],
    ("information", "information_schedule"): ["We are open from 9am to 10pm, 7 days a week.", "Our business hours are 9am to 5pm, Monday to Friday."],
    ("help", "help_with_finding"): ["Here are some hotels near the airport:...", "I can help you find a hotel that meets your needs."],
    ("help", "help_with_booking"): ["You can book a room on our website or by calling our reservation hotline.", "We also offer a loyalty program that gives you discounts on future bookings."],
    ("information", "ordering"): ["You can place an order on our website or by calling our order hotline.", "We also offer a loyalty program that gives you discounts on future orders."]
}

with gr.Blocks() as demo:
    chatbot = gr.Chatbot()
    msg = gr.Textbox()
    clear = gr.Button("Clear")

    def user(user_message, history):
        return "", history + [[user_message, None]]

    def bot(history):
        user_message = history[-1][0]

        intent = predict_intent(text=user_message)
        print('intent:', intent)
        # Random choice randomly chooses one of the options that matches the intent
     
        # generating a response with GPT if the main intent was 'privacy_policy' or 'legal_statement'
        use_gpt = intent[0] == 'privacy_policy' or intent[0] == 'legal_statement'

        response = gpt_model.answer_question(question=user_message) if use_gpt else 'No idea, bitch'

        # response = random.choice(response_map[intent])
        history[-1][1] = response
        # The sleep is to simulate a more natural conversation
        time.sleep(1)
        return history

    msg.submit(user, [msg, chatbot], [msg, chatbot], queue=False).then(
        bot, chatbot, chatbot
    )
    clear.click(lambda: None, None, chatbot, queue=False)

demo.launch()
